In [6]:
# Loading the reviews
import pandas as pd

labeled_train_review = pd.read_csv("./data/movie_reviews/labeledTrainData.tsv", header=0, delimiter="\t", quoting=3)
test_reviews = pd.read_csv("./data/movie_reviews/testData.tsv", header=0, delimiter="\t", quoting=3)
unlabeled_train_reviews = pd.read_csv("./data/movie_reviews/unlabeledTrainData.tsv", header=0,
                                      delimiter="\t", quoting=3)

# Verify the number of reviews that were read (100,000 in total)
print "labeled reviews: {}".format(labeled_train_review.shape)
print "test reviews: {}".format(test_reviews.shape)
print "unlabeled reviews: {}".format(unlabeled_train_reviews.shape)

print "\nlabeled reviews columns: {}".format(labeled_train_review.columns.values)


labeled reviews: (25000, 3)
test reviews: (25000, 2)
unlabeled reviews: (50000, 2)

labeled reviews columns: ['id' 'sentiment' 'review']


In [7]:
# Review to clean text
from bs4 import BeautifulSoup
import re
from nltk.corpus import stopwords

def review_to_wordlist(review, remove_stopwords=False):
    # Function to convert a document to a sequence of words,
    # optionally removing stop words.  Returns a list of words.
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(review, 'html5').get_text()
    #  
    # 2. Remove non-letters
    review_text = re.sub("[^a-zA-Z]", " ", review_text)
    #
    # 3. Convert words to lower case and split them
    words = review_text.lower().split()
    #
    # 4. Optionally remove stop words (false by default)
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if w not in stops]
    #
    # 5. Return a list of words
    return words


In [8]:
# Review to list of sentences

def review_to_sentences(review, sentence_tokenizer, remove_stopwords=False):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = sentence_tokenizer.tokenize(review.decode('utf-8').strip())
    #
    # 2. Loop over each sentence
    review_sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            review_sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))

    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return review_sentences

In [9]:
# Review to list of sentences (word2vec expect to get the sentences separated)

import nltk

# NLTK has a tokenizer allow us to split text by sentences
sentence_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

sentences = []

print "Parsing sentences from labeled training set"
for review in labeled_train_review["review"]:
    sentences += review_to_sentences(review, sentence_tokenizer)

print "Parsing sentences from unlabeled set"
for review in unlabeled_train_reviews["review"]:
    sentences += review_to_sentences(review, sentence_tokenizer)


Parsing sentences from training set


Parsing sentences from unlabeled set


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.msnbc.msn.com/id/4972055/site/newsweek/"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:219: UserWarning: ".." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://www.youtube.com/watch?v=a0KSqelmgN8"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


/usr/local/lib/python2.7/site-packages/bs4/__init__.py:282: UserWarning: "http://jake-weird.blogspot.com/2007/08/beneath.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


KeyboardInterrupt: 

In [ ]:
# Word2vec training
from gensim.models import word2vec
import logging

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

# Set values for various parameters
num_features = 300  # Word vector dimensionality                      
min_word_count = 40  # Minimum word count                        
num_workers = 4  # Number of threads to run in parallel
context = 10  # Context window size                                                                                    
downsampling = 1e-3  # Downsample setting for frequent words

model = word2vec.Word2Vec(sentences, workers=num_workers, size=num_features, min_count=min_word_count,
                          window=context, sample=downsampling)

# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
model.init_sims(replace=True)

# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
model_name = "{0}features_{1}minwords_{2}context".format(num_features, min_word_count, context)
model.save(model_name)

2018-03-03 11:59:37,611 : INFO : 'pattern' package not found; tag filters are not available for English


2018-03-03 11:59:37,614 : INFO : collecting all words and their counts


2018-03-03 11:59:37,615 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


2018-03-03 11:59:37,668 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types


2018-03-03 11:59:37,750 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


...

2018-03-03 12:00:53,472 : INFO : precomputing L2-norms of word weight vectors


2018-03-03 12:00:53,623 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None


2018-03-03 12:00:53,624 : INFO : not storing attribute vectors_norm


2018-03-03 12:00:53,626 : INFO : not storing attribute cum_table


2018-03-03 12:00:53,826 : INFO : saved 300features_40minwords_10context


In [1]:
# Here we can test our model

# Sentiment analysis
model.most_similar("good")

NameError: name 'model' is not defined